# Optimizing Neural Networks

To remind, necessary ingredients to train NN:
    * model
    * objective
    * optimizer
    
Today we will try to understand basics of optimization of neural networks, giving context for the last two lectures. Goal is to:
* Understand basics of generalization, and the difference between optimization and generalization (more on that in "Understanding generalization" lab)
* Understand impact of hyperparameters in SGD on:

  - generalization (lr, batch size)
  - speed of optimization (lr, momentum, batch size) 

References:
* Deep Learning book chapter on optimization: http://www.deeplearningbook.org/contents/optimization.html

# Setup

In [4]:
# Boilerplate code to get started

%load_ext autoreload
%autoreload 
%matplotlib inline

import json
import matplotlib as mpl
from src import fmnist_utils
from src.fmnist_utils import *

def plot(H):
    plt.title(max(H['test_acc']))
    plt.plot(H['acc'], label="acc")
    plt.plot(H['test_acc'], label="test_acc")
    plt.legend()

mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.figsize'] = (7, 7)
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['axes.labelsize'] = 12

(x_train, y_train), (x_test, y_test) = fmnist_utils.get_data()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/kg/miniconda3/envs/nn2018/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Exercise 1: optimization speed

Assuming fixed number of *epochs*, it is usually better to use either smaller batch size, or larger learning rate. Theoretical reason for it is not completely clear, so let's focus in this exercise on an empirical investigation.

Assume you are allowed to train the given network for 10 epochs. Answer the following questions:

* a) What was the optimal $\eta$ (assuming $S$=128 and $\mu$=0.9) for the final training accuracy?
* b) Did it also provide the best test accuracy? If yes, why (hint: consider if model is under or over-fitting)?
* c) What is the optimal $S$ (assuming $\eta$=0.1 and $\mu$=0.9) for the final training accuracy?
* d) Why is higher learning rate, or smaller batch size, optimizing faster? Give your best explanation (it can be hypothetical, there is no obvious theoretical answer)?

In [18]:
for lr in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]:
    model = build_mlp(784, 10, hidden_dims=[512])
    loss = torch.nn.CrossEntropyLoss(size_average=True)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
              x_test=x_test, y_test=y_test,
              optim=optimizer, batch_size=128, n_epochs=10)
    print("lr: ", lr, " train_acc: ", H['acc'][-1], " test_acc: ", H['test_acc'][-1])

 20%|██        | 2/10 [00:00<00:00, 11.36it/s]

lr:  0.0001  train_acc:  0.103  test_acc:  0.121


 20%|██        | 2/10 [00:00<00:00, 12.42it/s]

lr:  0.0005  train_acc:  0.474  test_acc:  0.475


 20%|██        | 2/10 [00:00<00:00, 12.53it/s]

...

lr:  0.05  train_acc:  0.869  test_acc:  0.769


 20%|██        | 2/10 [00:00<00:00, 12.64it/s]

lr:  0.1  train_acc:  0.876  test_acc:  0.775


100%|██████████| 10/10 [00:00<00:00, 11.86it/s]

lr:  0.5  train_acc:  0.675  test_acc:  0.591


In [19]:
for bs in [2, 4, 8, 16, 32, 64, 128, 256, 512]:
    model = build_mlp(784, 10, hidden_dims=[512])
    loss = torch.nn.CrossEntropyLoss(size_average=True)
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
              x_test=x_test, y_test=y_test,
              optim=optimizer, batch_size=bs, n_epochs=10)
    print("bs: ", bs, " train_acc: ", H['acc'][-1], " test_acc: ", H['test_acc'][-1])

  0%|          | 0/10 [00:00<?, ?it/s]

bs:  2  train_acc:  0.104  test_acc:  0.105


  0%|          | 0/10 [00:00<?, ?it/s]

bs:  4  train_acc:  0.134  test_acc:  0.133


  0%|          | 0/10 [00:00<?, ?it/s]

...

bs:  128  train_acc:  0.884  test_acc:  0.776


 20%|██        | 2/10 [00:00<00:00, 16.33it/s]

bs:  256  train_acc:  0.787  test_acc:  0.693


100%|██████████| 10/10 [00:00<00:00, 16.52it/s]

bs:  512  train_acc:  0.691  test_acc:  0.656


In [23]:
answers = {"a": "0.1", "b": "Yes. Might be underfitting?", "c": "128", "d": "Higher lr / lower batchsize is more noisy. Maybe it allows you to break out of flat areas / past saddle-points faster."}
json.dump(answers, open("7b_ex1.json", "w"))

# Exercise 2: generalization

Story with generalization is also unclear, but it is generally accepted that higher noise levels in SGD lead to better generalization. Think of noise in optimization (leading to low fidelity, as seen in lab 7a, for instance) as a close analog of typical regularizations (like dropout or batch normalization, that we will discuss next time).

Your task is to:

a) Check a range of LR and BS and find the best generalizing combination of LR and BS. What test accuracy were you able to achieve? What is the best LR and BS combination?

b) Answer the following question: Is stability correlated with using large LR or small BS. If yes, what is the intuitive reason for it? Feel free to give a hypothesis.

Hints:

* Make sure you achieve 100% training accuracy with each run, discard hyperparameters that are not achieving convergence.

Notes:

* Do not change the model in the starting code. It is on purpose a bit more complex MLP.

* You can measure stability by computing margin. This is implemented for you (using DeepFool method, https://arxiv.org/abs/1511.04599). Measuring margin is expensive, so recommended approach would be to compute it only on few final runs. 

In [21]:
for lr in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]:
    for bs in [2, 4, 8, 16, 32, 64, 128, 256, 512]:
        model = build_mlp(784, 10, hidden_dims=[512])
        loss = torch.nn.CrossEntropyLoss(size_average=True)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
                  x_test=x_test, y_test=y_test,
                  optim=optimizer, batch_size=bs, n_epochs=20)
        print("lr: ", lr, " bs: ", bs, " train_acc: ", H['acc'][-1], " test_acc: ", H['test_acc'][-1])

  0%|          | 0/20 [00:00<?, ?it/s]

lr:  0.0001  bs:  2  train_acc:  0.803  test_acc:  0.745


  0%|          | 0/20 [00:00<?, ?it/s]

lr:  0.0001  bs:  4  train_acc:  0.75  test_acc:  0.712


  0%|          | 0/20 [00:00<?, ?it/s]

...

lr:  0.5  bs:  128  train_acc:  0.71  test_acc:  0.634


 10%|█         | 2/20 [00:00<00:01, 17.42it/s]

lr:  0.5  bs:  256  train_acc:  0.815  test_acc:  0.735


100%|██████████| 20/20 [00:01<00:00, 18.94it/s]

lr:  0.5  bs:  512  train_acc:  0.682  test_acc:  0.645


In [39]:
answers = {"a": "Training till 100% training accuracy, the combination with the highest test accuracy was lr: 0.05, bs: 128 (with test acc of 0.793).", "b": "It does seem like it is (though with lr mattering more). If higher lr regularizes more, it makes sense that stability is higher as well.", "c": "There is no c?"}
json.dump(answers, open("7b_ex2.json", "w"))

## Stability measure

In 7a lab we discussed bias/variance view. Here, we will take a stability based view. To estimate stability, 
we will record maximum change in prediction when adding gaussian noise to examples. This is a very rudimentary
way to estimate geometric margin of the network, and we will talk more about this later.

In [8]:
from src.deepfool import measure_stability_deepfool

## Finding optimal $\eta$ and $S$

In [36]:
## Starting code

Hs = []
Lrs = [0.001, 0.005, 0.01, 0.05]
Margins = []
bss = [16, 32, 64, 128]

for lr in Lrs:
    for bs in bss:
        model = build_mlp(784, 10, hidden_dims=[100, 100, 100])
        loss = torch.nn.CrossEntropyLoss(size_average=True)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        H = train(loss=loss, model=model, x_train=x_train, y_train=y_train,
                  x_test=x_test, y_test=y_test,
                  optim=optimizer, batch_size=100, n_epochs=400)
        Margins.append(measure_stability_deepfool(model=model, 
                    x_train=x_train, y_train=y_train, loss=loss, N=1000))
        Hs.append(H)

100%|██████████| 400/400 [00:31<00:00, 13.08it/s]


In [37]:
i = 0
for lr in Lrs:
    for bs in bss:
        print("lr: ", lr, " bs: ", bs, " train_acc: ", Hs[i]['acc'][-1], " test_acc: ", Hs[i]['test_acc'][-1])
        print(Margins[i])
        i += 1

lr:  0.001  bs:  16  train_acc:  0.909  test_acc:  0.765
0.6658068
lr:  0.001  bs:  32  train_acc:  0.914  test_acc:  0.776
0.6743854
lr:  0.001  bs:  64  train_acc:  0.916  test_acc:  0.771
0.66958976
lr:  0.001  bs:  128  train_acc:  0.913  test_acc:  0.774
0.67680544
lr:  0.005  bs:  16  train_acc:  1.0  test_acc:  0.765
0.43906567
lr:  0.005  bs:  32  train_acc:  1.0  test_acc:  0.763
0.44927588
lr:  0.005  bs:  64  train_acc:  1.0  test_acc:  0.767
0.4437598
lr:  0.005  bs:  128  train_acc:  1.0  test_acc:  0.761
0.43897283
lr:  0.01  bs:  16  train_acc:  1.0  test_acc:  0.775
0.47367164
lr:  0.01  bs:  32  train_acc:  1.0  test_acc:  0.775
0.46811914
lr:  0.01  bs:  64  train_acc:  1.0  test_acc:  0.773
0.4699875
lr:  0.01  bs:  128  train_acc:  1.0  test_acc:  0.773
0.46465495
lr:  0.05  bs:  16  train_acc:  1.0  test_acc:  0.791
0.8310167
lr:  0.05  bs:  32  train_acc:  1.0  test_acc:  0.789
0.7979353
lr:  0.05  bs:  64  train_acc:  1.0  test_acc:  0.79
0.9257687
lr:  0.05  bs: